In [ ]:
import os
import matplotlib.pyplot as plt
from PIL import Image, ImageEnhance
import pandas as pd

# Making the pairs for synthetic as well

In [ ]:
f = os.listdir("/content/drive/MyDrive/Colab_Notebooks/GenAI4V Project 2/sketch2code_dataset_v1_cleaned")
f_s = os.listdir("/content/drive/MyDrive/Colab_Notebooks/GenAI4V Project 2/screenshots_synthetic")

In [ ]:
html_count = 0
sketch_count = 0

for f_ in f:
    if '.html' in f_:
        html_count += 1
    if ('_' in f_) and ('.png' in f_):
        sketch_count += 1

print(html_count)
print(sketch_count)

484
731


In [ ]:
sketches = []
originals = []

In [ ]:
for f_ in f:
    if ('_' in f_) and ('.png' in f_):
        sketches.append(f_)
        originals.append(f_[:f_.index('_')] + '.png')

In [ ]:
for f_ in f_s:
    if ('_' in f_) and ('.png' in f_):
        originals.append(f_)
        sketches.append(f_[:f_.index('_')] + '_0' + '.png')

In [ ]:
data = {'sketches': sketches, 'originals': originals}
df = pd.DataFrame(data)

In [ ]:
df.to_csv('/content/drive/MyDrive/Colab_Notebooks/GenAI4V Project 2/Sketch2Webpage_Synthetic.csv')

In [ ]:
df

,sketches,originals
0,13004_1.png,13004.png
1,14378_0.png,14378.png
2,12730_0.png,12730.png
3,15265_0.png,15265.png
4,512_1.png,512.png
...,...,...
1882,15845_0.png,15845_red.png
1883,1800_0.png,1800_dark.png
1884,1800_0.png,1800_red.png
1885,15845_0.png,15845_blue.png


# Creating the dataset

In [ ]:
src_dir = "/content/drive/MyDrive/Colab_Notebooks/GenAI4V Project 2/sketch2code_dataset_v1_cleaned"
dst_dir = "/content/drive/MyDrive/Colab_Notebooks/GenAI4V Project 2/sketch2code_dataset_v1_cleaned"

os.makedirs(dst_dir, exist_ok=True)

valid_exts = (".png", ".jpg", ".jpeg")

for fname in os.listdir(src_dir):
    if not fname.lower().endswith(valid_exts):
        continue

    src_path = os.path.join(src_dir, fname)

    # Force .jpeg extension
    base_name = os.path.splitext(fname)[0]
    dst_fname = base_name + ".jpg"
    dst_path = os.path.join(dst_dir, dst_fname)

    try:
        img = Image.open(src_path).convert("RGB")  # removes alpha if PNG
        img.save(dst_path, "JPEG", quality=95)

        print(f"Saved: {dst_fname}")

    except Exception as e:
        print(f"Failed: {fname} | {e}")

Saved: 14781.jpg
Saved: 13004_1.jpg
Saved: 14378_0.jpg
Saved: 12730_0.jpg
Saved: 15265_0.jpg
Saved: 512_1.jpg
Saved: 77_0.jpg
Saved: 13657_0.jpg
Saved: 10685.jpg
Saved: 7713_1.jpg
Saved: 12910.jpg
Saved: 5257.jpg
Saved: 12634_2.jpg
Saved: 10903.jpg
Saved: 8486_0.jpg
Saved: 15713_0.jpg
Saved: 5485.jpg
Saved: 17739.jpg
Saved: 12943_0.jpg
Saved: 2504.jpg
Saved: 10691.jpg
Saved: 11202.jpg
Saved: 10564_0.jpg
Saved: 13211.jpg
Saved: 13775_0.jpg
Saved: 12058_0.jpg
Saved: 188.jpg
Saved: 341_0.jpg
Saved: 1352.jpg
Saved: 14378.jpg
Saved: 13365.jpg
Saved: 2303.jpg
Saved: 6159.jpg
Saved: 3747_1.jpg
Saved: 4618_0.jpg
Saved: 1967_0.jpg
Saved: 6018_0.jpg
Saved: 9719_1.jpg
Saved: 792_0.jpg
Saved: 3747.jpg
Saved: 13753_1.jpg
Saved: 6946_0.jpg
Saved: 2830_0.jpg
Saved: 7864.jpg
Saved: 2883.jpg
Saved: 4202.jpg
Saved: 3380.jpg
Saved: 10297.jpg
Saved: 62.jpg
Saved: 1673_2.jpg
Saved: 8745_0.jpg
Saved: 6946_1.jpg
Saved: 4405.jpg
Saved: 6159_0.jpg
Saved: 9912_0.jpg
Saved: 15864_1.jpg
Saved: 17268_1.jpg
Saved: 

# Uploading the synthetic datatset

In [ ]:
import os
from datasets import Dataset, Features, Image as HFImage, Value

In [ ]:
pairs = df

In [ ]:
thermal_dir = "/content/drive/MyDrive/Colab_Notebooks/GenAI4V Project 2/sketch2code_dataset_v1_cleaned"
rgb_dir = "/content/drive/MyDrive/Colab_Notebooks/GenAI4V Project 2"

In [ ]:
def gen():
    # for idx, fname in enumerate(names):
    for idx, row in pairs.iterrows():
        # print(row)
        thermal_path = os.path.join(thermal_dir, row['sketches']).replace(".png", ".jpeg")

        rgb_img = row['originals']
        if '_' in rgb_img:
          rgb_path = os.path.join(os.path.join(rgb_dir, "screenshots_synthetic"), rgb_img).replace(".png", ".jpg")
        else:
          rgb_path = os.path.join(os.path.join(rgb_dir, "sketch2code_dataset_v1_cleaned"), rgb_img).replace(".png", ".jpg")

        yield {
            "conditioning_image": thermal_path,
            "image": rgb_path,
            "prompt": "",
        }

features = Features({
    "conditioning_image": HFImage(),
    "image": HFImage(),
    "prompt": Value("string"),
})

dataset = Dataset.from_generator(gen, features=features)

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
paths = [
    "/content/drive/MyDrive/Colab_Notebooks/GenAI4V Project 2/sketch2code_dataset_v1_cleaned/10018_0.jpeg",
    "/content/drive/MyDrive/Colab_Notebooks/GenAI4V Project 2/sketch2code_dataset_v1_cleaned/1002_0.jpeg"
]

for p in paths:
    img = Image.open(p).convert("RGB")
    img = img.resize((384, 384), Image.BILINEAR)
    img.save(p)